In [ ]:
!pip install 'scanpy == 1.9.1'

In [ ]:
import os, sys
import scanpy as sc
import shutil

ModuleNotFoundError: ignored

In [ ]:
def Correlated_Genes_Unique_Validity_Identifier(timepoint_1_clust_genes, timepoint_2_clust_genes, timepoint_pair_annadata_tuple, clust_1, clust_2, gene):

  all_clusters_timepoint_1 = timepoint_1_clust_genes.keys()
  all_clusters_timepoint_2 = timepoint_2_clust_genes.keys()

  timepoint_1_curr_gene_unique = False
  timepoint_2_curr_gene_unique = False

  timepoint_1_curr_all_clusts_greater = []
  timepoint_2_curr_all_clusts_greater = []

  mean_exp_percentage_clust_1 = float((timepoint_pair_annadata_tuple[0].raw[timepoint_pair_annadata_tuple[0].obs['E_I_M'].isin([clust_1]),:][:,gene].X > 0).mean(0)[0][0])
  mean_exp_percentage_clust_2 = float((timepoint_pair_annadata_tuple[1].raw[timepoint_pair_annadata_tuple[1].obs['E_I_M'].isin([clust_2]),:][:,gene].X > 0).mean(0)[0][0])

  for clusters in all_clusters_timepoint_1:
    if clusters != clust_1:
      t_1_mean_exp_percentage_other_clust = float((timepoint_pair_annadata_tuple[0].raw[timepoint_pair_annadata_tuple[0].obs['E_I_M'].isin([clusters]),:][:,gene].X > 0).mean(0)[0][0])
      if (mean_exp_percentage_clust_1 - t_1_mean_exp_percentage_other_clust) > .1:
        timepoint_1_curr_all_clusts_greater.append(clusters)
  for clusters in all_clusters_timepoint_2:
    if clusters != clust_2:
      t_2_mean_exp_percentage_other_clust = float((timepoint_pair_annadata_tuple[1].raw[timepoint_pair_annadata_tuple[1].obs['E_I_M'].isin([clusters]),:][:,gene].X > 0).mean(0)[0][0])
      if (mean_exp_percentage_clust_2 - t_2_mean_exp_percentage_other_clust) > .1:
        timepoint_2_curr_all_clusts_greater.append(clusters)

  if len(timepoint_1_curr_all_clusts_greater) == len(all_clusters_timepoint_1)-1:
    timepoint_1_curr_gene_unique = True
  if len(timepoint_2_curr_all_clusts_greater) == len(all_clusters_timepoint_2)-1:
    timepoint_2_curr_gene_unique = True

  return timepoint_1_curr_gene_unique, timepoint_2_curr_gene_unique

In [ ]:
def Correlated_Gene_Mean_Tuple_Constructor(timepoint_pair_name_tuple, all_timepoint_unique_genes_dict, clust_1, clust_2, gene):
  clust_1_gene_mean_tuples = all_timepoint_unique_genes_dict.get(timepoint_pair_name_tuple[0]).get(clust_1)
  clust_2_gene_mean_tuples = all_timepoint_unique_genes_dict.get(timepoint_pair_name_tuple[1]).get(clust_2)
  clust_1_mean = [gene_tup[1] for gene_tup in clust_1_gene_mean_tuples if gene_tup[0]==gene][0]
  clust_2_mean = [gene_tup[1] for gene_tup in clust_2_gene_mean_tuples if gene_tup[0]==gene][0]

  if clust_1_mean == float("inf"):
    clust_1_mean = 1000000
  if clust_2_mean == float("inf"):
    clust_2_mean = 1000000

  total_mean = (clust_1_mean+clust_2_mean) / 2
  correlated_gene_mean_tuple = (gene, total_mean)

  return correlated_gene_mean_tuple

In [ ]:
def Unique_Genes_Only_Filterer(all_timepoint_unique_genes_dict):

  all_timepoint_unique_genes_ONLY_dict = {}

  for timepoint in all_timepoint_unique_genes_dict:

    top_genes_cluster_dict = {}
    unique_dicts = all_timepoint_unique_genes_dict.get(timepoint)
    all_clusters_curr_timepoint_True_Relative_Distinct_Unique_Genes = {}

    for clust in unique_dicts:
      temp_clust_genes = []
      gene_unique_tuples = unique_dicts.get(clust)
      list_of_unique_candidate_distinct_genes = []

      for gene_tup in gene_unique_tuples:

        list_of_unique_candidate_distinct_genes.append(gene_tup[0])

      list_of_unique_candidate_distinct_genes = list_of_unique_candidate_distinct_genes
      top_genes_cluster_dict[clust] = list_of_unique_candidate_distinct_genes

    all_timepoint_unique_genes_ONLY_dict[timepoint] = top_genes_cluster_dict

  return all_timepoint_unique_genes_ONLY_dict

In [ ]:
def Correlated_Unique_Genes_Constructor(timepoint_pair_name_tuples, timepoint_pair_anndata_tuples, all_timepoint_unique_genes_dict):
  print('Average Runtime: 23 minutes \n')

  all_timepoint_correlated_unique_genes_dict = {}
  all_timepoint_unique_genes_ONLY_dict = Unique_Genes_Only_Filterer(all_timepoint_unique_genes_dict)

  for timepoint_pair_name_tuple, timepoint_pair_annadata_tuple in zip(timepoint_pair_name_tuples, timepoint_pair_anndata_tuples):

    timepoint_pair_correlated_unique_genes_dict = {}
    timepoint_pair_correlated_unique_genes_with_scores_dict = {}

    timepoint_1_clust_genes = all_timepoint_unique_genes_ONLY_dict.get(timepoint_pair_name_tuple[0])
    timepoint_2_clust_genes = all_timepoint_unique_genes_ONLY_dict.get(timepoint_pair_name_tuple[1])

    for clust_1 in timepoint_1_clust_genes:

      clust_1_genes = set(timepoint_1_clust_genes.get(clust_1))

      for clust_2 in timepoint_2_clust_genes:

        clust_2_genes = set(timepoint_2_clust_genes.get(clust_2))
        correlated_genes = clust_1_genes.intersection(clust_2_genes)
        correlated_gene_mean_tuples = []

        for x, gene in enumerate(correlated_genes):
          print('\rWorking on     '+str(timepoint_pair_name_tuple)+'   Current Timepoint: ' + str(timepoint_pair_name_tuple[0])+'    Cluster:'+clust_1+'    '+str(timepoint_pair_name_tuple[1])+'    Cluster:'+clust_2+ '    Gene: '+ gene+'  '+ str(x + 1) + ':' + str(len(correlated_genes)), end='')
          timepoint_1_curr_gene_unique, timepoint_2_curr_gene_unique = Correlated_Genes_Unique_Validity_Identifier(timepoint_1_clust_genes, timepoint_2_clust_genes, timepoint_pair_annadata_tuple, clust_1, clust_2, gene)

          if timepoint_1_curr_gene_unique and timepoint_2_curr_gene_unique:
            correlated_gene_mean_tuple = Correlated_Gene_Mean_Tuple_Constructor(timepoint_pair_name_tuple, all_timepoint_unique_genes_dict, clust_1, clust_2, gene)
            correlated_gene_mean_tuples.append(correlated_gene_mean_tuple)

        correlated_gene_mean_tuples_sorted = sorted(correlated_gene_mean_tuples, key=lambda x: int(x[1]), reverse = True)
        correlated_genes_sorted = [x[0] for x in correlated_gene_mean_tuples_sorted]

        if correlated_genes_sorted:
          timepoint_pair_correlated_unique_genes_dict['Cluster_'+clust_1+'__'+'Cluster_'+clust_2] = correlated_genes_sorted

    all_timepoint_correlated_unique_genes_dict[timepoint_pair_name_tuple[0]+'_'+timepoint_pair_name_tuple[1]] = timepoint_pair_correlated_unique_genes_dict
    print('\n')

  return all_timepoint_correlated_unique_genes_dict

In [ ]:
def Correlated_Unique_Genes_Text_File_Writer(curr_timepoint_path, timepoint_correlated_pair_dict, timepoint_pair_name):

  corr_txt_dir = curr_timepoint_path+'/'+timepoint_pair_name+'.txt'
  corr_txt_dir = open(corr_txt_dir,"w")

  corr_txt_dir.write(timepoint_pair_name+' Correlated Unique Genes:'+'\n\n')

  for correlation in timepoint_correlated_pair_dict:

    corr_txt_dir.write(correlation+'\n\n')

    corr_unique_genes_list = [x for x in timepoint_correlated_pair_dict.get(correlation)]

    for gene in corr_unique_genes_list:

      corr_txt_dir.write(gene+'\n')

    corr_txt_dir.write('\n\n')

  corr_txt_dir.close()

In [ ]:
def time_path(timepoint):

  time = ''

  if '/' in timepoint:
    time = timepoint.replace('/','_')

  else:
    time = timepoint

  return time

In [ ]:
def Correlated_Unique_Genes_Dotplot_Constructor(correlated_unique_genes_path, timepoint_pair_name_tuples, timepoint_pair_anndata_tuples, all_timepoint_correlated_unique_genes_dict):

  for timepoint_pair_anndata_tuple, timepoint_pair_name_tuple in zip(timepoint_pair_anndata_tuples, timepoint_pair_name_tuples):

    time_1 = time_path(timepoint_pair_name_tuple[0])
    time_2 = time_path(timepoint_pair_name_tuple[1])

    timepoint_pair_path_name = time_1+'_'+time_2
    timepoint_pair_dict_name = timepoint_pair_name_tuple[0] + '_' + timepoint_pair_name_tuple[1]

    timepoint_correlated_pair_dict = all_timepoint_correlated_unique_genes_dict.get(timepoint_pair_dict_name)

    curr_timepoint_path = correlated_unique_genes_path + '/'+ timepoint_pair_path_name

    if not os.path.exists(curr_timepoint_path):
       os.mkdir(curr_timepoint_path)

    Correlated_Unique_Genes_Text_File_Writer(curr_timepoint_path, timepoint_correlated_pair_dict, timepoint_pair_path_name)

    for cluster_pairs in timepoint_correlated_pair_dict:

      curr_corr_clust_path = curr_timepoint_path + '/' + cluster_pairs

      if not os.path.exists(curr_corr_clust_path):
        os.mkdir(curr_corr_clust_path)

      correlated_genes = timepoint_correlated_pair_dict.get(cluster_pairs)

      sc.settings.figdir = curr_corr_clust_path

      t1_clust = cluster_pairs.split('__')[0].split('_')[1]
      t2_clust = cluster_pairs.split('__')[1].split('_')[1]

      t1_title = timepoint_pair_name_tuple[0]+' vs. '+timepoint_pair_name_tuple[1] + '\n'+cluster_pairs+ '\n\n'+timepoint_pair_name_tuple[0] + '\n' +'Cluster: ' + cluster_pairs.split('__')[0].split('_')[1]
      t2_title = timepoint_pair_name_tuple[0]+' vs. '+timepoint_pair_name_tuple[1] + '\n'+cluster_pairs+ '\n\n'+timepoint_pair_name_tuple[1] + '\n' +'Cluster: ' + cluster_pairs.split('__')[1].split('_')[1]

      sc.pl.dotplot(timepoint_pair_anndata_tuple[0], correlated_genes[0:30], title= t1_title, mean_only_expressed=True, groupby='E_I_M', save= time_1+'_'+ t1_clust +'.png');
      sc.pl.dotplot(timepoint_pair_anndata_tuple[1], correlated_genes[0:30], title= t2_title, mean_only_expressed=True, groupby='E_I_M', save= time_2+'_'+ t2_clust +'.png');